In [2]:
import os
import cv2
import numpy as np
from ultralytics import SAM
from sklearn.model_selection import train_test_split
import json
from pycocotools import mask as maskUtils
from tensorflow.keras.utils import Sequence

E0000 00:00:1737268627.097584    2223 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737268627.100500    2223 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
extract_dir = './roof_data/'

In [4]:
class DataGenerator(Sequence):
    def __init__(self, data_dir, batch_size=8, shuffle=True):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_list = self._get_file_list()
        self.on_epoch_end()

    def _get_file_list(self):
        file_list = []
        for root, _, files in os.walk(self.data_dir):
            for file in files:
                if file.endswith('.json'):
                    file_list.append(os.path.join(root, file))
        return file_list

    def __len__(self):
        return int(np.floor(len(self.file_list) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.file_list[index * self.batch_size:(index + 1) * self.batch_size]
        images, masks = self._load_data(batch_files)
        return images, masks

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.file_list)

    def _load_data(self, batch_files):
        images = []
        masks = []
        for json_path in batch_files:
            with open(json_path, 'r') as f:
                data = json.load(f)
                image_path = os.path.join(os.path.dirname(json_path), data['image']['file_name'])
                if os.path.exists(image_path):
                    image = cv2.imread(image_path)
                    if image is not None:
                        images.append(image / 255.0)  # Normalize the image
                        # Create an empty mask
                        mask = np.zeros((data['image']['height'], data['image']['width']), dtype=np.uint8)
                        for annotation in data['annotations']:
                            rle = annotation['segmentation']
                            binary_mask = maskUtils.decode(rle)
                            mask = np.maximum(mask, binary_mask)
                        masks.append(mask / 255.0)  # Normalize the mask
                    else:
                        print(f"Failed to read image: {image_path}")
                else:
                    print(f"File does not exist: {image_path}")
        return np.array(images), np.array(masks)

In [ ]:
# train_data_dir = os.path.join(extract_dir, 'train')
# val_data_dir = os.path.join(extract_dir, 'valid')

# train_images, train_masks = load_data(train_data_dir)
# val_images, val_masks = load_data(val_data_dir)


In [9]:
train_data_dir = os.path.join(extract_dir, 'train')
val_data_dir = os.path.join(extract_dir, 'valid')

train_generator = DataGenerator(train_data_dir, batch_size=8, shuffle=True)
val_generator = DataGenerator(val_data_dir, batch_size=8, shuffle=False)

In [11]:
model = SAM('./sam2.1_b.pt')

In [13]:
model.train(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Adjust the number of epochs as needed
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    learning_rate=0.001  # Adjust the learning rate as needed
)


KeyError: 'model'